In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.model_selection import train_test_split

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
train_path= os.path.join('..','input','intel-image-classification','seg_train','seg_train')

In [8]:
test_path= os.path.join('..','input','intel-image-classification','seg_test','seg_test')

In [9]:
train_data= image_dataset_from_directory(train_path,image_size=(150,150),
                                         label_mode='categorical',
                                         batch_size=64)

test_data = image_dataset_from_directory(test_path,image_size=(150,150),
                                          label_mode='categorical',
                                          batch_size=64)

In [10]:
print('Data Class Indicies:', test_data.class_names)

In [11]:
plt.figure(figsize=(15,13))

class_names= train_data.class_names

for images,labels in train_data.take(1):
    for i in range(30):
        ax=plt.subplot(6,6,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')

In [21]:
early_stopping_monitor = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5)

best_model = ModelCheckpoint('bestmodel.hdf5', monitor='val_accuracy', save_best_only=True)

In [22]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(150,150,3)),
    Conv2D(32, (3,3), activation='relu'),  
    MaxPooling2D(2,2),
    
    Conv2D(32, (3,3), activation='relu'),
    Conv2D(32, (3,3), activation='relu'),    
    MaxPooling2D(2,2),
    
    Conv2D(32, (3,3), activation='relu'),    
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(6, activation='softmax')
])

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [24]:
model.fit(train_data, validation_data= test_data, epochs=30, 
         callbacks=[best_model, reduce_lr_on_plateau, early_stopping_monitor])

In [25]:
# accuracy of the model

plt.figure(figsize=(12, 6))
plt.plot(model.history.history["accuracy"])
plt.plot(model.history.history["val_accuracy"])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("ACCURACY OF MODEL")
plt.legend(['training_accuracy', 'validation_accuracy'])
plt.show()

In [26]:
# loss of the model
plt.figure(figsize=(12, 6))
plt.plot(model.history.history["loss"])
plt.plot(model.history.history["val_loss"])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("LOSS OF MODEL")
plt.legend(['training_loss', 'validation_loss'])
plt.show()

In [27]:
model.load_weights('bestmodel.hdf5')
model.evaluate(test_data);

In [28]:
#Visualize 30 of the predicted data with green label for correct predictions
#and red label vise versa.
plt.figure(figsize=(15,15))
for images, labels in test_data.take(2):
    for i in range(35):
        prediction = model.predict(images[i].numpy().reshape(1,150,150,3))
        plt.subplot(7, 7, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.grid(False)
        xlabel = plt.xlabel(class_names[prediction.argmax()])
        if (prediction.argmax() == labels[i].numpy().argmax()):
            xlabel.set_color("green")
        else:
            xlabel.set_color("red")
plt.show()